## Simple parametric modelling of radar backscatter from vegetation & soil
- **Author**: Felix Zaussinger
- **Date**: 01.04.2019 (with updates from 21.11.2020)
- **Acknowledgements**: based on invaluable course material created by Wolfgang Wagner  & Raphael Quast (TU Wien), augmented with helpful comments given by Raphael.

### Definition of the backscattering coefficient

The scattering coefficient $\sigma_0$ is defined as

$$\sigma_0 = \frac{d\sigma}{dA}$$

where $d\sigma$ is the differential scattering cross section and $dA$ the differential unit area. In the microwave remote sensing literature, synonyms for $\sigma_0$ are __sigma nought__ and __normalised radar cross section__. Furthermore, in the case of a monostatic radar, $\sigma_0$ is called the __backscattering coefficient__.

Important assumptions for the specification of $\sigma_0$ include: 

1. The observed area consists of many individual point scatters with random locations and amplitudes.
2. No single scatterer's amplitude dominates the superposition of individual signals.

### Modeling the interaction of radar waves with planet earth's surface

On an abstract level, we need to consider the following scattering components:

1. $\sigma^{0}_{v}$ ... Volume scattering from vegetation canopies
2. $\sigma^{0}_{s}$ ... Surface scattering from the underlying soil surface
3. $\sigma^{0}_{sv}$ ... Volume-surface scattering interactions

Canopy-level backscatter can then be expressed as a linear combination of scattering components

$$\sigma^{0}_{can} = \sigma^{0}_{v} + \sigma^{0}_{s} + \sigma^{0}_{sv}$$

One approach to model these interactions is the __Cloud Model__ (Attema, Ulaby 1978). The model consists of three layers:

**Air**
- Downwelling radiation is not attenuated until reaching the air-vegetation boundary.


**Vegetation**
- Individual scatteres are uniformly distibuted throughout a homogenous volume such as air. 
- The vegetation layer can be thought of as a cloud which droplets are held in place by vegetative matter. 
- The incoming radiation is exponentially attenuated as a function of extinction coefficient $k_e$ and distance $z$.


**Soil**
- The downwelling radiation which has penetrated the vegetation layer is partly reflected backwards as a function of the Fresnel coefficient $r_I$.
- $\sigma^{0}_{s}$ is governed by the reflection properties of the soil and the loss in the vegetation layer.

**Important**

When using jupyter lab, call %matplotlib widget prior to importing pyplot to enable "interactiveness".

In [1]:
%matplotlib widget

**Packages**

In [2]:
from __future__ import print_function
import numpy as np
import matplotlib.pyplot as plt
from scipy import constants
import ipywidgets as widgets
from IPython.display import display

**Parameters**

In [3]:
# incidence angles
thetas = np.linspace(20, 75, 100)
thetas_rad = np.deg2rad(thetas)

### 1. Bare-soil model (Champion 1996)

Function of soil moisture and incidence angle. Multiplicative effect in the linear domain, additive effect in the decibel-domain. In a simple linear model, the radar response from bare soil is related to **volumetric soil moisture content** of the first soil centimeters $m_v$ (nowadays this is more often specified as $SMC$ or $SWC$)

$$ \sigma^{0}_{s}(dB) = C_{\theta} + D * m_v $$

where $D$ is the sensitivity of the radar signal to variations in $m_v$. Note that $C_{\theta}$ is the value of $\sigma^{0}_{s}$ for a perfectly dry soil (i.e., $m_v=0~g~cm^{-3}$). It is approximated through a simple cosine model

$$ C_{\theta} = C1 + C2 * \cos(\theta)^{C3}$$

where the "C-parameters" account for the dependence of backscatter on **incidence angle**:

- C1: $\sigma^{0}_{s}$ for $\theta=90°$
- C2: $\sigma^{0}_{s}$ over $\theta=[0°-60°]$
- C3: shape factor of the cosine model (connected to angular decrease)

In [4]:
def bare_soil_model(theta, mv):
    """Returns sigma0_soil in dB given the incidence angle theta and volumetric soil water content mv."""
    
    # Cosine model parameters for 5.3 GHz and HH pol from Champion (1996)
    C1, C2, C3 = (-29.2, 27.2, 2.8)
    
    # Soil moisture sensitivity D
    D = 0.28 # 0.32 also feasible

    C_theta = C1 + C2 * np.cos(theta) ** C3
    return C_theta + D * mv

In [5]:
# mv: from 0.02 (dry) to 0.3 (wet)
smc = 1

sigma0_bare_soil = bare_soil_model(thetas_rad, smc)

fig, ax = plt.subplots(figsize=(5,5))
ax.grid(color='#EEEEEE', linewidth=1, linestyle='solid')

line, = ax.plot(thetas, sigma0_bare_soil)

ax.set_title('Bare soil model')
ax.set_xlabel(r'Incidence angle $\theta$ (°)')
ax.set_ylabel(r'$\sigma^{0}_{s}$ (dB)')
ax.set_ylim(-30, 5)


def update(smc):
    sigma0_bare_soil = bare_soil_model(thetas_rad, smc)
    line.set_ydata(sigma0_bare_soil)

smc_slider = widgets.FloatSlider(min=0, max=40, step=1, value=0, description=r'$SMC (g/cm^{3})$')
widgets.interact(update, smc=smc_slider)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=0.0, description='$SMC (g/cm^{3})$', max=40.0, step=1.0), Output()), _…

<function __main__.update(smc)>

### 2. Cloud model (Attema, Ulaby 1978)

**Reference:** *Attema, E. P. W., and Fawwaz T. Ulaby. "Vegetation modeled as a water cloud." Radio science 13.2 (1978): 357-364.*


"Because the microwave dielectric constant of dry vegetative matter is much smaller (by an order of magnitude or more) than the dielectric constant of water, and because a vegetation canopy is usually composed of more than 99% air by volume, it is proposed that the canopy can be modeled as a water cloud whose droplets are held in place by the vegetative matter. Such a model was developed assuming that the canopy "cloud" contains identical water droplets randomly distributed within the canopy. By integrating the scattering and attenuation cross-section contributions of N droplets per unit volume over the signal pathlength through the canopy, an expression was derived for the backscattering coefficient as a function of three target parameters: volumetric moisture content of the soil, volumetric water content of the vegetation, and plant height. Regression analysis of the model predictions against scattering data acquired over a period of four months at several angles of incidence (0°-70°) and frequencies (8-18 GHz) for HH and VV polarizations yields correlation coefficients that range from .7 to .99 depending on frequency, polarization, and crop type. The corresponding standard errors of estimate range from 1.1 to 2.6 dB."

**Model specification**

Single scattering albedo: $\alpha = \frac{k_s}{k_e}$

Two-way attenuation factor: $\gamma = e^{-\frac{2\tau}{\cos(\theta)}}$

Parameters: $\theta$, $k_s$, $k_e$, $\tau$

*Individual contributions:*

- Volume component $\sigma^{0}_{v} = \frac{\alpha}{2} (1-\gamma^2)$ 
- Attenuated soil component: $\gamma^2 \sigma^{0}_{soil}$
- Soil-volume interaction component: often not included
    - complex relationship (difficult to parametrise in a robust way)
    - limited contribution to the overall signal (somewhat neglectable second-order effect)

In [6]:
def dB2lin(x):
    return np.power(10, x/10)

def lin2dB(x):
    return 10* np.log10(x)

def cloud_model(theta, smc, alpha, tau):
    """"""
    # Two-way attenuation factor
    y2 = np.exp(-2*tau/np.cos(theta))
    
    # compute bare soil contribution and convert back to linear scale
    sigma0_soil_dB = bare_soil_model(theta, smc)
    sigma0_soil_linear = dB2lin(sigma0_soil_dB)
    
    # compute canopy backscattering coefficient using the cloud model and convert to dB scale
    sigma0_can = ((3*alpha*np.cos(theta))/4 * (1-y2)) + (y2 * sigma0_soil_linear)
    
    # return total, volume, soil and soil-volume components
    return {"Canopy": lin2dB(sigma0_can), # total canopy backscatter,
            "Volume": lin2dB(((3*alpha*np.cos(theta))/4 * (1-y2))), # volume term
            "Soil": lin2dB(y2 * sigma0_soil_linear)} # soil term

In [7]:
# params
tau = 0.1
alpha = 0.1
smc = 1

# calculate backscatter curves
cloud_model_terms = cloud_model(thetas_rad, smc, alpha, tau) # canopy backscatter

# create figure
fig, ax = plt.subplots(figsize=(5,5))
ax.grid(color='#EEEEEE', linewidth=1, linestyle='solid')

# plot lines
sigma0_canopy_plot, = ax.plot(thetas, cloud_model_terms['Canopy'], label=r'$\sigma^{0}_{can}$', linestyle='--')
sigma0_volume_plot, = ax.plot(thetas, cloud_model_terms['Volume'], label=r'$\sigma^{0}_{v}$')
sigma0_soil_plot, = ax.plot(thetas, cloud_model_terms['Soil'], label=r'$\sigma^{0}_{s}$')

# labelling
ax.set_title('Cloud model')
ax.set_xlabel(r'Incidence angle $\theta$ (°)')
ax.set_ylabel(r'$\sigma^{0}$ (dB)')
ax.set_ylim(-40, 10)
ax.legend()

# interactive plot
def update(tau, alpha, smc):
    # update model
    cloud_model_terms = cloud_model(thetas_rad, smc, alpha, tau) # canopy backscatter
    
    # update figure
    sigma0_canopy_plot.set_ydata(cloud_model_terms['Canopy'])
    sigma0_volume_plot.set_ydata(cloud_model_terms['Volume'])
    sigma0_soil_plot.set_ydata(cloud_model_terms['Soil'])

# sliders
tau_slider = widgets.FloatSlider(min=0.01, max=1, step=0.05, value=0.1, description=r'$\tau$ (Np)')
alpha_slider = widgets.FloatSlider(min=0.01, max=1, step=0.05, value=0.1, description=r'$\alpha$')
smc_slider = widgets.FloatSlider(min=0, max=40, step=1, value=0, description=r'$SMC (g(cm^{3})$')

widgets.interact(update, tau=tau_slider, alpha=alpha_slider, smc=smc_slider)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=0.1, description='$\\tau$ (Np)', max=1.0, min=0.01, step=0.05), FloatS…

<function __main__.update(tau, alpha, smc)>